<a href="https://colab.research.google.com/github/MiguelEgidoMorales/UFV-visualizacion/blob/main/ejercicio_de_clase/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### EJERCICIO CLASE

In [ ]:

import pandas as pd

df = pd.read_csv('/content/datos_ejercicio_ventas.csv',sep=";")

df.head()


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,"7,54356E+15"
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,"5,60031E+15"
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15"
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,"3,63225E+15"
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,"3,96176E+16"


**1.   NºActuals y Forecasts**

In [ ]:
num_actuals = df[df['SCENARIO'] == 'actual'].shape[0]


num_forecasts = df[df['SCENARIO'] == 'AI_forecast'].shape[0]

print(f"Número de Actuals: {num_actuals}")
print(f"Número de Forecasts: {num_forecasts}")

Número de Actuals: 900
Número de Forecasts: 17766


**3.  Nº Países y de productos**

In [ ]:

num_paises = df['COUNTRY'].nunique()
print(f'Número de países: {num_paises}')


num_productos = df['SUBBRAND'].nunique()
print(f'Número de productos: {num_productos}')

Número de países: 9
Número de productos: 6


**4.  Histórico de actuals/forecasts**
  - Primer dato de ventas real hasta el último
  - Primer dato de veentas forecast hasta el último

In [ ]:
import pandas as pd

# Cargar el archivo CSV

# Convertir YEAR y MONTH a enteros
df['YEAR'] = df['YEAR'].astype(int)
df['MONTH'] = df['MONTH'].astype(int)

# Ordenar los datos por año y mes para encontrar los primeros y últimos valores
df = df.sort_values(by=['YEAR', 'MONTH'])

# Filtrar los datos reales (actuals) y obtener las primeras y últimas fechas
actuals_data = df[df['SCENARIO'] == 'actual']
first_actual = actuals_data.iloc[0]
last_actual = actuals_data.iloc[-1]

# Filtrar los datos de predicción (forecasts) y obtener las primeras y últimas fechas
forecast_data = df[df['SCENARIO'] == 'AI_forecast']
first_forecast = forecast_data.iloc[0]
last_forecast = forecast_data.iloc[-1]

# Mostrar los resultados
print("Primer dato de ventas real (actual): Año {}, Mes {}".format(first_actual['YEAR'], first_actual['MONTH']))
print("Último dato de ventas real (actual): Año {}, Mes {}".format(last_actual['YEAR'], last_actual['MONTH']))
print("Primer dato de ventas forecast: Año {}, Mes {}".format(first_forecast['YEAR'], first_forecast['MONTH']))
print("Último dato de ventas forecast: Año {}, Mes {}".format(last_forecast['YEAR'], last_forecast['MONTH']))

Primer dato de ventas real (actual): Año 2023, Mes 1
Último dato de ventas real (actual): Año 2024, Mes 8
Primer dato de ventas forecast: Año 2023, Mes 1
Último dato de ventas forecast: Año 2025, Mes 5


# PRACTICA 1




###**PREPARACIÓN DATOS:**




In [342]:
import pandas as pd

df = pd.read_csv('/content/datos_ejercicio_ventas.csv',delimiter=',')
df.to_csv('/content/datos_ejercicio_ventas.csv', sep=';', index=False)
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


In [343]:
type(df['AMOUNT'].iloc[0])

numpy.float64

In [344]:

# Crear la columna 'Fecha' usando directamente las columnas 'YEAR' y 'MONTH'
df['Fecha'] = pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))

# Eliminar las columnas 'YEAR' y 'MONTH' si ya no se necesitan
df.drop(columns=['YEAR', 'MONTH'], inplace=True)

# Mostrar el DataFrame
df.head()


,COUNTRY,SUBBRAND,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Fecha
0,Portugal,Lipton (L3),AI_forecast,AI_P02F,2023.0,754356.237194,2023-12-01
1,Great Britain,Lipton (L3),AI_forecast,AI_P10F,2023.0,560030.558029,2023-12-01
2,Spain,Pepsi Max (L3),AI_forecast,AI_P09F,2023.0,88501.980847,2023-12-01
3,Great Britain,7up (L3),AI_forecast,AI_P10F,2023.0,363224.511516,2024-12-01
4,Hungary,Lipton (L3),AI_forecast,AI_P03F,2023.0,396176.120491,2023-09-01


In [ ]:
type(df['AMOUNT'].iloc[0])

str

In [ ]:
type(df['AMOUNT'].iloc[0])

str

In [345]:
# Crear un diccionario que mapea los valores de FORECAST a los meses
forecast_to_month = {
    'AI_P02F': 1,  # Enero
    'AI_P03F': 2,  # Febrero
    'AI_P04F': 3,  # Marzo
    'AI_P05F': 4,  # Abril
    'AI_P06F': 5,  # Mayo
    'AI_P07F': 6,  # Junio
    'AI_P08F': 7,  # Julio
    'AI_P09F': 8,  # Agosto
    'AI_P10F': 9,  # Septiembre
    'AI_P11F': 10, # Octubre
    'AI_P12F': 11, # Noviembre
    'AI_PF': 12    # Diciembre
}

# Usar map() para transformar los valores de la columna 'FORECAST' en meses
df['MONTH_FROM_FORECAST'] = df['FORECAST'].map(forecast_to_month)

# Mostrar el DataFrame para verificar



df.head()

,COUNTRY,SUBBRAND,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT,Fecha,MONTH_FROM_FORECAST
0,Portugal,Lipton (L3),AI_forecast,AI_P02F,2023.0,754356.237194,2023-12-01,1.0
1,Great Britain,Lipton (L3),AI_forecast,AI_P10F,2023.0,560030.558029,2023-12-01,9.0
2,Spain,Pepsi Max (L3),AI_forecast,AI_P09F,2023.0,88501.980847,2023-12-01,8.0
3,Great Britain,7up (L3),AI_forecast,AI_P10F,2023.0,363224.511516,2024-12-01,9.0
4,Hungary,Lipton (L3),AI_forecast,AI_P03F,2023.0,396176.120491,2023-09-01,2.0


In [346]:
type(df['AMOUNT'].iloc[0])

numpy.float64

In [347]:
df["FECHA_FORECAST"] = pd.to_datetime(
    df[["MONTH_FROM_FORECAST", "FORECAST_YEAR"]]
    .assign(DAY=1)
    .rename(columns={"MONTH_FROM_FORECAST": "MONTH", "FORECAST_YEAR": "YEAR"})
)
df = df.drop(columns=['MONTH_FROM_FORECAST', 'FORECAST_YEAR','FORECAST'])
df.head()

,COUNTRY,SUBBRAND,SCENARIO,AMOUNT,Fecha,FECHA_FORECAST
0,Portugal,Lipton (L3),AI_forecast,754356.237194,2023-12-01,2023-01-01
1,Great Britain,Lipton (L3),AI_forecast,560030.558029,2023-12-01,2023-09-01
2,Spain,Pepsi Max (L3),AI_forecast,88501.980847,2023-12-01,2023-08-01
3,Great Britain,7up (L3),AI_forecast,363224.511516,2024-12-01,2023-09-01
4,Hungary,Lipton (L3),AI_forecast,396176.120491,2023-09-01,2023-02-01


In [348]:
df = df.drop_duplicates()

df.head()

,COUNTRY,SUBBRAND,SCENARIO,AMOUNT,Fecha,FECHA_FORECAST
0,Portugal,Lipton (L3),AI_forecast,754356.237194,2023-12-01,2023-01-01
1,Great Britain,Lipton (L3),AI_forecast,560030.558029,2023-12-01,2023-09-01
2,Spain,Pepsi Max (L3),AI_forecast,88501.980847,2023-12-01,2023-08-01
3,Great Britain,7up (L3),AI_forecast,363224.511516,2024-12-01,2023-09-01
4,Hungary,Lipton (L3),AI_forecast,396176.120491,2023-09-01,2023-02-01


<class 'numpy.float64'>


In [349]:
# Mostrar solo las filas duplicadas en todas las columnas
duplicados = df[df.duplicated()]

# Contar el número de filas duplicadas
count_duplicados = len(duplicados)

print("Total de filas duplicadas:", count_duplicados)
print(duplicados)


Total de filas duplicadas: 0
Empty DataFrame
Columns: [COUNTRY, SUBBRAND, SCENARIO, AMOUNT, Fecha, FECHA_FORECAST]
Index: []


In [350]:
# Filtrar los datos para obtener los actuales (SCENARIO = 'actual')
df_actuals = df[df['SCENARIO'] == 'actual'].reset_index(drop=True)

# Filtrar los datos para obtener las predicciones (SCENARIO = 'AI_forecast')
df_ia = df[df['SCENARIO'] == 'AI_forecast']
df_actuals = df_actuals.drop(columns=['SCENARIO','FECHA_FORECAST'])
df_ia = df_ia.drop(columns=['SCENARIO'])

In [351]:
print("DataFrame de datos actuales:")
df_actuals.head()

DataFrame de datos actuales:


,COUNTRY,SUBBRAND,AMOUNT,Fecha
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Portugal,7up (L3),2.934971e+05,2023-03-01
2,Portugal,7up (L3),3.484466e+05,2023-10-01
3,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
4,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


In [352]:
print("\nDataFrame de predicciones:")
df_ia.head()


DataFrame de predicciones:


,COUNTRY,SUBBRAND,AMOUNT,Fecha,FECHA_FORECAST
0,Portugal,Lipton (L3),754356.237194,2023-12-01,2023-01-01
1,Great Britain,Lipton (L3),560030.558029,2023-12-01,2023-09-01
2,Spain,Pepsi Max (L3),88501.980847,2023-12-01,2023-08-01
3,Great Britain,7up (L3),363224.511516,2024-12-01,2023-09-01
4,Hungary,Lipton (L3),396176.120491,2023-09-01,2023-02-01


In [353]:
num_paises = df_actuals['COUNTRY'].nunique()
print(f'Número de países: {num_paises}')


num_productos = df_actuals['SUBBRAND'].nunique()
print(f'Número de productos: {num_productos}')

Número de países: 9
Número de productos: 6


In [354]:
df_actuals.shape
df_actuals.dtypes


,0
COUNTRY,object
SUBBRAND,object
AMOUNT,float64
Fecha,datetime64[ns]


In [355]:
print(df_actuals['AMOUNT'].isna().sum())

#897

0


In [378]:
df_PA_act = df_actuals.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
df_PA_act = df_PA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})  # Renombrar la columna 'AMOUNT' a 'TOTAL_AMOUNT'



In [379]:
print(df_PA_act)

         COUNTRY  TOTAL_AMOUNT
0          Czech  3.535164e+07
1        Denmark  5.659668e+07
2  Great Britain  3.347786e+08
3        Hungary  4.153991e+07
4          Italy  4.345404e+07
5    Netherlands  6.395943e+07
6         Norway  5.121406e+07
7       Portugal  3.488807e+07
8          Spain  8.131266e+06


In [384]:
df_FA_act = df_actuals.groupby('Fecha')['AMOUNT'].sum().reset_index()
df_FA_act = df_FA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})


In [387]:
df_FA_act.head()

,Fecha,TOTAL_AMOUNT
0,2023-01-01,2.808880e+07
1,2023-02-01,2.610437e+07
2,2023-03-01,3.593163e+07
3,2023-04-01,3.085725e+07
4,2023-05-01,3.690245e+07


In [396]:
df_MA_act = df_actuals.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
df_MA_act = df_MA_act.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})

In [397]:
df_MA_act.head()

,SUBBRAND,TOTAL_AMOUNT
0,7up (L3),1.876851e+07
1,7up Free (L3),3.427630e+07
2,Lipton (L3),7.286515e+07
3,Mountain Dew (L3),7.474391e+06
4,Pepsi Max (L3),3.966987e+08


## 1.Como se distribuyen las ventas realizadas:


1.   En cada país
2.   En cada mes y año
3.   En cada marca



###**1. En cada país:**




In [376]:
# Crear la gráfica de barras
fig = px.bar(df_PA_act,
             x='COUNTRY',  # Países en el eje X
             y='TOTAL_AMOUNT',   # Montos en el eje Y
             title='Distribución de ventas en cada País',  # Título de la gráfica
             labels={'TOTAL_AMOUNT': 'CANTIDAD EN L', 'COUNTRY': 'Países'},  # Etiquetas de los ejes
             color='COUNTRY',  # Puedes asignar colores diferentes a cada país
             color_discrete_sequence=px.colors.qualitative.Set1  # Color discreto para las barras
            )

# Actualizar el formato del eje Y para mostrar valores en formato científico
fig.update_layout(
    yaxis_tickformat='.2e'  # El formato '.2e' muestra los números en notación científica con 2 decimales
)

# Mostrar la gráfica
fig.show()

###**2.En cada mes y año**

In [393]:
# Crear la gráfica de barras
fig = px.bar(df_FA_act,
             x='Fecha',  # Fechas en el eje X
             y='TOTAL_AMOUNT',  # Montos en el eje Y
             title='Distribución de ventas realizadas en cada Mes y año',  # Título de la gráfica
             labels={'TOTAL_AMOUNT': 'Cantidad en L', 'Fecha': 'Fecha'},  # Etiquetas de los ejes
            )

# Actualizar el formato del eje Y para mostrar los valores en formato científico
fig.update_layout(
    yaxis_tickformat='.2e',  # Mostrar en formato científico con 2 decimales
)

# Mostrar la gráfica
fig.show()

###**3.En cada marca**

In [399]:
fig = px.pie(df_MA_act,
             names='SUBBRAND',  # Categorías en el gráfico (SUBBRAND)
             values='TOTAL_AMOUNT',  # Valores correspondientes a cada categoría (TOTAL_AMOUNT)
             title='Distribución de ventas realizadas en cada marca',  # Título del gráfico
             color='SUBBRAND',  # Asigna colores diferentes a cada categoría
             color_discrete_sequence=px.colors.qualitative.Set3  # Paleta de colores discreta
            )

# Mostrar la gráfica
fig.show()

##2.Cual es la tendencia y estacionalidad de:


1.   Todas las ventas del país con menos ventas
2.   La marca con más ventas




###**1.Todas las ventas del país con menos ventas**

In [400]:
# Ordenar el DataFrame de menor a mayor en base a la columna 'TOTAL_AMOUNT'
df_PA_sorted_act = df_PA_act.sort_values(by='TOTAL_AMOUNT', ascending=True)

# Mostrar el DataFrame ordenado
print(df_PA_sorted_act)

         COUNTRY  TOTAL_AMOUNT
8          Spain  8.131266e+06
7       Portugal  3.488807e+07
0          Czech  3.535164e+07
3        Hungary  4.153991e+07
4          Italy  4.345404e+07
6         Norway  5.121406e+07
1        Denmark  5.659668e+07
5    Netherlands  6.395943e+07
2  Great Britain  3.347786e+08


In [411]:
df_espana = df_actuals[df_actuals['COUNTRY'] == 'Spain']  # Filtrar para España
df_espana = df_espana.drop(columns=['SUBBRAND'])  # Eliminar la columna 'SUBBRAND'
df_espana = df_espana.groupby(['COUNTRY', 'Fecha'])['AMOUNT'].sum().reset_index()  # Agrupar por 'COUNTRY' y 'Fecha'
df_espana = df_espana.rename(columns={'AMOUNT': 'TOTAL_AMOUNT'})  # Renombrar la columna 'AMOUNT' a 'TOTAL_AMOUNT'
print(df_espana)  # Imprimir el DataFrame resultante


   COUNTRY      Fecha   TOTAL_AMOUNT
0    Spain 2023-01-01  325272.938930
1    Spain 2023-02-01  295588.379605
2    Spain 2023-03-01  399710.273110
3    Spain 2023-04-01  328035.257874
4    Spain 2023-05-01  360751.156280
5    Spain 2023-06-01  439198.586290
6    Spain 2023-07-01  496545.236120
7    Spain 2023-08-01  564336.120820
8    Spain 2023-09-01  420041.980591
9    Spain 2023-10-01  468425.841760
10   Spain 2023-11-01  418006.199049
11   Spain 2023-12-01  394147.408590
12   Spain 2024-01-01  349696.533142
13   Spain 2024-02-01  352899.035652
14   Spain 2024-03-01  332109.256233
15   Spain 2024-04-01  376678.368897
16   Spain 2024-05-01  356759.011786
17   Spain 2024-06-01  408967.746367
18   Spain 2024-07-01  506234.362782
19   Spain 2024-08-01  537862.283643


In [415]:

fig = px.line(df_espana,
              x='Fecha',  # Fechas en el eje X
              y='TOTAL_AMOUNT',  # Montos en el eje Y
              title='Monto por Fecha para España',  # Título de la gráfica
              labels={'AMOUNT': 'Monto en Euros', 'Fecha': 'Fecha'},  # Etiquetas de los ejes
              line_group='COUNTRY',  # Agrupar por país (aunque en este caso solo hay un país)
              color='COUNTRY'  # Asigna colores a las líneas (aunque solo hay una línea para España)
             )

# Mostrar la gráfica
fig.show()

###**2.La marca con más ventas**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Filtrar el DataFrame solo para la marca 'Pepsi Max (L3)'
df_pepsi_max = df[df['SUBBRAND'] == 'Pepsi Max (L3)']

# Agrupar por YEAR y MONTH para sumar AMOUNT en cada combinación de mes y año
df_pepsi_max_grouped = df_pepsi_max.groupby(['YEAR', 'MONTH'], as_index=False)['AMOUNT'].sum()

# Crear la columna 'Fecha' combinando YEAR y MONTH
df_pepsi_max_grouped['Fecha'] = pd.to_datetime(df_pepsi_max_grouped['YEAR'].astype(str) + '-' + df_pepsi_max_grouped['MONTH'].astype(str) + '-01')

# Establecer 'Fecha' como índice
df_pepsi_max_grouped.set_index('Fecha', inplace=True)

# Descomposición de la serie temporal de las ventas de Pepsi Max (L3)
decomposition = seasonal_decompose(df_pepsi_max_grouped['AMOUNT'], model='multiplicative', period=12)  # Asumiendo datos mensuales

# Graficar los componentes
fig = plt.figure(figsize=(10, 8))

# Graficar tendencia
plt.subplot(411)
plt.plot(decomposition.trend)
plt.title('Tendencia - Pepsi Max (L3)')

# Graficar estacionalidad
plt.subplot(412)
plt.plot(decomposition.seasonal)
plt.title('Estacionalidad - Pepsi Max (L3)')

# Graficar la serie original
plt.subplot(413)
plt.plot(df_pepsi_max_grouped['AMOUNT'])
plt.title('Serie Original - Pepsi Max (L3)')

# Ajustar la disposición de los gráficos
plt.tight_layout()
plt.show()



##3.Cuales son las predicciones hechas en España y como de buenas son.